In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv

class SpatioTemporalGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, time_steps):
        super(SpatioTemporalGNN, self).__init__()
        
        # Graph Convolution Layers
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)
        
        # Temporal Model
        self.gru = nn.GRU(hidden_channels, hidden_channels, batch_first=True)
        
        # Fully Connected Output Layer
        self.fc = nn.Linear(hidden_channels, out_channels)
        
        # Number of time steps
        self.time_steps = time_steps

    def forward(self, x, edge_index, batch):
        """
        x: Node features of shape [num_nodes, in_channels, time_steps]
        edge_index: Graph edges of shape [2, num_edges]
        batch: Batch indices for mini-batch processing
        """
        # Initialize hidden states for GRU
        h = None
        
        # List to store predictions
        outputs = []
        
        for t in range(self.time_steps):
            # Extract node features at time step t
            x_t = x[:, :, t]
            
            # GCN Layers
            x_t = F.relu(self.gcn1(x_t, edge_index))
            x_t = F.relu(self.gcn2(x_t, edge_index))
            
            # Reshape for GRU (batch_size, seq_len=1, hidden_channels)
            x_t = x_t.unsqueeze(1)
            
            # Temporal Learning with GRU
            x_t, h = self.gru(x_t, h)
            
            # Output prediction
            out = self.fc(x_t[:, -1, :])
            outputs.append(out)
        
        # Stack outputs along time dimension
        outputs = torch.stack(outputs, dim=-1)
        return outputs


In [2]:
import pandas as pd
import numpy as np

# Define latitude-longitude pairs
locations = [
    (37.7749, -122.4194),  # San Francisco
    (34.0522, -118.2437),  # Los Angeles
    (40.7128, -74.0060),   # New York
    (41.8781, -87.6298),   # Chicago
    (29.7604, -95.3698)    # Houston
]

# Generate a time series for 30 days
days = pd.date_range(start="2024-01-01", periods=30, freq="D")

# Simulate incident counts
data = []
for loc_id, (lat, lon) in enumerate(locations):
    for day in days:
        incident_count = np.random.poisson(5)  # Simulated counts using Poisson distribution
        data.append({
            "location_id": loc_id,
            "latitude": lat,
            "longitude": lon,
            "date": day,
            "incident_count": incident_count
        })

# Convert to DataFrame
df = pd.DataFrame(data)

# Preview the dataset
print(df.head(5))


   location_id  latitude  longitude       date  incident_count
0            0   37.7749  -122.4194 2024-01-01               9
1            0   37.7749  -122.4194 2024-01-02               3
2            0   37.7749  -122.4194 2024-01-03               1
3            0   37.7749  -122.4194 2024-01-04               3
4            0   37.7749  -122.4194 2024-01-05               7


In [5]:
import torch
from sklearn.metrics.pairwise import haversine_distances
from math import radians

# Parameters
time_steps = 5  # Number of past days to use as features
num_locations = len(locations)

# Step 1: Prepare Node Features
node_features = []
for loc_id in range(num_locations):
    location_data = df[df["location_id"] == loc_id]
    incident_series = location_data["incident_count"].values
    
    # Create sliding windows
    features = [
        incident_series[i:i + time_steps]
        for i in range(len(incident_series) - time_steps)
    ]
    node_features.append(features)

# Stack features for all nodes
x = torch.tensor(node_features, dtype=torch.float32).permute(1, 0, 2)  # Shape: [time_steps, num_nodes, num_features]

# Step 2: Create Edge Index (Spatial Proximity)
coords = np.radians([(lat, lon) for lat, lon in locations])
dist_matrix = haversine_distances(coords)  # Distance in radians

# Define edges based on nearest neighbors
edge_index = []
threshold = 0.5  # Approx 50 km in radians
for i in range(num_locations):
    for j in range(num_locations):
        if i != j and dist_matrix[i, j] < threshold:
            edge_index.append([i, j])

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Preview prepared tensors
print("Node Features (x):", x.shape)  # [num_samples, num_nodes, time_steps]
print("Edge Index:", edge_index.shape)  # [2, num_edges]


Node Features (x): torch.Size([25, 5, 5])
Edge Index: torch.Size([2, 16])


In [6]:
# Initialize Model
model = SpatioTemporalGNN(
    in_channels=time_steps,  # Number of input features
    hidden_channels=16,      # Hidden layer size
    out_channels=1,          # Forecast one value (incident count)
    time_steps=time_steps
)

# Forward Pass
output = model(x, edge_index, batch=None)
print("Output Shape:", output.shape)  # Should be [num_samples, num_nodes, out_channels]


Output Shape: torch.Size([25, 1, 5])


In [7]:
# Example: Last 7 days of incident counts for 5 nodes
recent_data = torch.tensor([
    [6, 5, 7, 8, 9, 10, 11],  # Node 1
    [3, 4, 5, 6, 7, 8, 9],    # Node 2
    [10, 9, 8, 7, 6, 5, 4],   # Node 3
    [1, 2, 3, 4, 5, 6, 7],    # Node 4
    [2, 3, 4, 5, 6, 7, 8],    # Node 5
], dtype=torch.float32).unsqueeze(2)  # Shape: [num_nodes, time_steps, 1]

# Reshape for GNN input: [num_nodes, in_channels, time_steps]
x = recent_data.permute(0, 2, 1)  # Shape: [5, 1, 7]


In [8]:
# Example: Connections between 5 nodes
edge_index = torch.tensor([
    [0, 1, 2, 3, 4, 0],  # Source nodes
    [1, 2, 3, 4, 0, 4],  # Target nodes
], dtype=torch.long)  # Shape: [2, num_edges]


In [9]:
# Assume the model is trained
model.eval()  # Set the model to evaluation mode

# Forward pass
with torch.no_grad():
    forecast = model(x, edge_index, batch=None)  # Shape: [num_nodes, out_channels]

# Output forecast
print("Forecasted Incident Counts:")
print(forecast)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x1 and 5x16)